In [35]:
import ants


In [ ]:
# create T2 with nulled CSF (fake FLAIR)
t2_cerebra = ants.image_read('atlas/mni_icbm152_nlin_sym_09c/mni_icbm152_t2_tal_nlin_sym_09c.nii')
t1_cerebra = ants.image_read('atlas/mni_icbm152_nlin_sym_09c/mni_icbm152_t1_tal_nlin_sym_09c.nii')
cerebra_brainmask = ants.image_read('atlas/mni_icbm152_nlin_sym_09c/mni_icbm152_t1_tal_nlin_sym_09c_mask.nii')

t2_cerebra[cerebra_brainmask < 1] = 0
t1_cerebra[cerebra_brainmask < 1] = 0

t2_jhu_brain = ants.image_read('atlas/JHU_T2_brain.nii.gz')

mytx = ants.registration(fixed=t2_jhu_brain, moving=t2_cerebra, type_of_transform='SyN')
t1_cerebra_in_jhu = ants.apply_transform(fixed=t2_jhu_brain, moving=t1_cerebra, transformlist=mytx['fwdtransforms'])
t1_cerebra_in_jhu.to_filename('atlas/t1_mni_to_jhu.nii.gz')

t2_cerebra_csf = ants.image_read('atlas/mni_icbm152_nlin_sym_09c/mni_icbm152_csf_tal_nlin_sym_09c.nii')
t2_cerebra *= (1- t2_cerebra_csf)
t2_cerebra.to_filename('atlas/mni_icbm152_nlin_sym_09c_t2_csfnull.nii.gz')

In [36]:
# register MNI152nonlinear to MNI152 linear space (there are slight differences)
# then use this registration to register the Harvard-Oxford atlas (which is in linear space) to nonlinear space  
# (the fake FLAIR is in nonlinear space)

linear_atlas = ants.image_read('atlas/mni_icbm152_lin_nifti/icbm_avg_152_t1_tal_lin.nii')
linear_atlas_mask = ants.image_read('atlas/mni_icbm152_lin_nifti/icbm_avg_152_t1_tal_lin_mask.nii')
linear_atlas[linear_atlas_mask == 0] = 0

nlin_atlas = ants.image_read('atlas/mni_icbm152_nlin_sym_09c/mni_icbm152_t1_tal_nlin_sym_09c.nii')
nlin_atlas_mask = ants.image_read('atlas/mni_icbm152_nlin_sym_09c/mni_icbm152_t1_tal_nlin_sym_09c_mask.nii')
nlin_atlas[nlin_atlas_mask == 0] = 0

harvard_oxford = ants.image_read('atlas/Harvard-Oxford cortical and subcortical structural atlases/HarvardOxford-cort_and_sub-maxprob-thr25-1mm.nii.gz_1.nii.gz')

registration = ants.registration(fixed=linear_atlas, moving=nlin_atlas, type_of_transform='SyNAggro')
harvard_oxford_to_nlin = ants.apply_transforms(fixed=nlin_atlas, moving=harvard_oxford, transformlist=registration['invtransforms'], interpolator='genericLabel')
harvard_oxford_to_nlin[nlin_atlas_mask == 0] = 0
harvard_oxford_to_nlin.to_file('atlas/harvard_oxford_to_nlin.nii.gz')